# Twitter Crawler

In [10]:
import sys
!{sys.executable} -m pip install pymongo

    100% |████████████████████████████████| 348kB 2.9MB/s eta 0:00:01
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [71]:
# Import Packages
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pymongo
import numpy as np
import pandas as pd

In [72]:
# Target URL
url = "https://www.twitter.com/Nike"

In [73]:
# Using Chrome WebDriver
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [74]:
# Beautiful Soup scrolls down to the bottom of the page
lastHeight = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        break
    lastHeight = newHeight

soupfetch = BeautifulSoup(driver.page_source, "html.parser")

In [112]:
# Extract Information from URL
data = pd.DataFrame({'tweet': [], 'hashtag': [], 'mentions': [], 'no_of_replies': [], 'no_of_likes': [], 'no_of_retweets': [], 'date': [], 'time': []})

In [121]:
import csv
# Open/Create a file to append data
csvFile = open('niketweets1.csv', 'w')
#Use csv Writer
csvWriter = csv.writer(csvFile)

In [122]:
twitterwebdata = soupfetch.find_all('div', {'class': 'content'})
for twitter_tweet in twitterwebdata:
    try:
        tweet = twitter_tweet.find('p', {'class': 'TweetTextSize'}).text
        #print(tweet)
    except:
        pass
    
    hashtag = []
    for tag in twitter_tweet.find_all('a', {'class': 'twitter-hashtag'}):
        hashtag.append(tag.text[1:])
    if hashtag == []:
        hashtag = np.NaN
    #print(hashtag)
    
    mentions = []
    for a in twitter_tweet.find_all('a', {'class': 'twitter-atreply'}):
        try:
            mentions.append(a['href'][1:])
        except:
            pass
    if mentions == []:
        mentions = np.NaN
    #print(mentions)
    
    thumbs = twitter_tweet.find_all('span', \
                        {'class': 'ProfileTweet-actionCountForPresentation'})
    try:
        re = thumbs[0].text
        if re[-1] == 'K':
            re = int(float(re[:-1]) * 1000)
        else:
            re = int(float(re.encode))
    except:
        pass
    #print(re)
    
    try:
        like = thumbs[3].text
        if like[-1] == 'K':
            like = int(float(like[:-1])*1000)
        else:
            like = int(float(like))
    except:
        pass
    #print(like)
    
    try:
        retweet = thumbs[2].text
        if retweet[-1] == 'K':
            retweet = int(float(retweet[:-1])*1000)
        else:
            retweet = int(float(retweet))
    except:
        pass
    #print(retweet)
    try:
        date = twitter_tweet.find("a",attrs={"class":"tweet-timestamp js-permalink js-nav js-tooltip"})["title"].split('-')[1]
    except:
        date = None
        pass
    #print(date)
    
    try:
        time = twitter_tweet.find("a",attrs={"class":"tweet-timestamp js-permalink js-nav js-tooltip"})["title"].split('-')[0]
    except:
        time = None
        pass
    #print(time)
    
    #tweet_feed = {'tweet': tweet, 'hashtag': hashtag, 'mentions': mentions, \
                  #'no_of_replies': re, 'no_of_likes': like, 'no_of_retweets': retweet, 'date': date, 'time': time}
    #data = data.append([pd.DataFrame(tweet_feed)], ignore_index=True)
    #pd.Series(data).to_frame()
    csvWriter.writerow([tweet, hashtag, mentions, re, like, retweet, date, time])